In [3]:
print("ok")

ok


In [4]:
%pwd

'e:\\mydata\\Medical APP\\Medical_GenAI_App\\research'

In [3]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
#extract data from PDF

def load_pdf_file(data):
    '''Load file from directory, data=directory path,
    glob=file type, loader_cls=loader to use
    Usage: load_pdf_file(data=Data/)
    '''
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()

    return documents

In [5]:
%pwd

'e:\\mydata\\Medical APP\\Medical_GenAI_App\\research'

In [6]:
import os
os.chdir('../')

In [7]:
%pwd

'e:\\mydata\\Medical APP\\Medical_GenAI_App'

In [8]:
extacted_data = load_pdf_file(data='Data/')

In [9]:
#Split data into text chunks
def text_split(extracted_data):
    '''split documents into chunks. Example: text_split(data)'''
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [10]:
text_chunks = text_split(extacted_data)
print("Length of text chunks:", len(text_chunks))

Length of text chunks: 5860


In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

In [12]:
#Download embedding model from HuggingFaceEndpoint
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_huggingface_embeddings()

c:\Users\HiTECH\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
import os
os.environ['PINECONE_API_KEY'] = os.environ.get('PINECONE_API_KEY')

In [16]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [20]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [22]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [23]:
docsearch

In [24]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [25]:
retrieved_docs = retriever.invoke("what is Acromegaly and gigantism?")

In [26]:
retrieved_docs


[Document(id='b78762e4-64d3-483c-adea-3d9cf7e7574d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 45.0, 'page_label': '46', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Whitehouse Station, NJ: Merck Research Laboratories,\n1997.\nLarsen, D. E., ed. Mayo Clinic Family Health Book.New York:\nWilliam Morrow and Co., Inc., 1996.\nJohn T. Lohr, PhD\nAcromegaly and gigantism\nDefinition\nAcromegaly is a disorder in which the abnormal\nrelease of a particular chemical from the pituitary gland\nin the brain causes increased growth in bone and soft tis-\nsue, as well as a variety of other disturbances throughout\nthe body. This chemical released from the pituitary gland'),
 Document(id='4fe290b8-f42c-46a6-8307-f107f1e7f9bb', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 47.0,

In [27]:
groq_api_key=os.environ.get("GROQ_API_KEY")

In [28]:
from langchain_groq import ChatGroq

# llm=ChatGroq(groq_api_key=groq_api_key,
#              model_name="Llama3-8b-8192")

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="Llama3-8b-8192",
    temperature=0.7,        # Controls randomness (0.0 for deterministic, 1.0 for max creativity)
    max_tokens=500          # Maximum tokens in the response
)

In [29]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human',"{input}")
    ]
)


In [30]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [31]:
response = rag_chain.invoke({'input': "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder in which the abnormal release of a particular chemical from the pituitary gland in the brain causes increased growth in bone and soft tissue, as well as a variety of other disturbances throughout the body. This condition occurs when the abnormality occurs after bone growth stops, and it is relatively rare, occurring in approximately 50 out of every one million people.


In [34]:
print(response)

{'input': 'what is Acromegaly and gigantism?', 'context': [Document(id='ad096f8e-980d-4826-af8a-369ddb05df9b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 45.0, 'page_label': '46', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Whitehouse Station, NJ: Merck Research Laboratories,\n1997.\nLarsen, D. E., ed. Mayo Clinic Family Health Book.New York:\nWilliam Morrow and Co., Inc., 1996.\nJohn T. Lohr, PhD\nAcromegaly and gigantism\nDefinition\nAcromegaly is a disorder in which the abnormal\nrelease of a particular chemical from the pituitary gland\nin the brain causes increased growth in bone and soft tis-\nsue, as well as a variety of other disturbances throughout\nthe body. This chemical released from the pituitary gland'), Document(id='029c4aab-7a8a-45ac-925d-340bab6953ce', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPD

In [36]:
#out of context question to check hallucination
response = rag_chain.invoke({'input': "what is machine learning?"})
print(response["answer"])

I don't know. The provided context does not mention machine learning. It appears to be discussing human memory systems, neurodegenerative disease, and key terms related to biology and psychology.
